In [1]:
# Import libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

In [27]:
news_df = pd.read_csv("tweets/yahoo_financial_news_sentiments.csv")
news_df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1356793542228774916,1356793542228774916,2021-02-03 10:36:03 Malay Peninsula Standard Time,2021-02-03,10:36:03,800,19546277,yahoofinance,Yahoo Finance,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1356784358254907393,1356784358254907393,2021-02-03 09:59:33 Malay Peninsula Standard Time,2021-02-03,09:59:33,800,19546277,yahoofinance,Yahoo Finance,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1356744021360730113,1356744021360730113,2021-02-03 07:19:16 Malay Peninsula Standard Time,2021-02-03,07:19:16,800,19546277,yahoofinance,Yahoo Finance,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1356734439104126977,1356734439104126977,2021-02-03 06:41:11 Malay Peninsula Standard Time,2021-02-03,06:41:11,800,19546277,yahoofinance,Yahoo Finance,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1356728921287520256,1356728921287520256,2021-02-03 06:19:16 Malay Peninsula Standard Time,2021-02-03,06:19:16,800,19546277,yahoofinance,Yahoo Finance,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [28]:
# news_df.set_index('date', inplace=True)
df = news_df[['tweet','date','time']]
df.head()

,tweet,date,time
0,Alibaba plans up to $5 billion U.S.-dollar bon...,2021-02-03,10:36:03
1,Treasury's Yellen to call regulator meeting on...,2021-02-03,09:59:33
2,Amazon CEO Jeff Bezos is stepping down — here'...,2021-02-03,07:19:16
3,Amazon history: How Jeff Bezos started with 20...,2021-02-03,06:41:11
4,Jeff Bezos email to employees: 'Keep inventing...,2021-02-03,06:19:16


In [29]:
df.dropna(inplace=True)

<ipython-input-29-c64f9f573c18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [30]:
#Cleaning the tweets

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def clean(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'pic.twitter\S+', ' ', text)
    text = decontracted(text)
    text = re.sub(r'\(([^)]+)\)', " ", text)
    text = text.replace('etmarkets', ' ').replace('marketupdates', ' ').replace('newsalert', ' ').replace('ndtv', ' ').replace('moneycontrol', ' ').replace('here is why', ' ')
    text = text.replace('marketsupdate', ' ').replace('biznews', ' ').replace('click here', ' ').replace('live updates', ' ').replace('et now', ' ')
    text = re.sub(r'[^a-zA-Z ]+', ' ', text)
    text = re.sub(r' \w{1,2}_', ' ', text)
    text = re.sub('\s+',' ', text)
    return text

In [31]:
for i, row in df.iterrows():
    print(row[0])
    tweet_val = decontracted(row[0])
    tweet_val = clean(tweet_val)
    df.at[i,'tweet'] = tweet_val

Alibaba plans up to $5 billion U.S.-dollar bond issuance: term sheet  https://t.co/INE0Mull8o  https://t.co/M1FyGCzrWy
Treasury's Yellen to call regulator meeting on GameStop volatility: Reuters  https://t.co/pGJFYizU5E  https://t.co/BkYtUEq5up
Amazon CEO Jeff Bezos is stepping down — here's why Amazon stock isn't crashing  https://t.co/14gOL3fKKH by @BrianSozzi  https://t.co/rfKmrIsPyN
Amazon history: How Jeff Bezos started with 20 mail order products and then dominated e-commerce  https://t.co/EOaH127VlA by @ewolffmann
Jeff Bezos email to employees: 'Keep inventing, and don't despair when at first the idea looks crazy'  https://t.co/UVve1YRlGr by @JuliaLaRoche  https://t.co/PQCVMFdpJL
.@rookisaacman is buying a SpaceX flight and piloting it into space. "It's gonna be the first all-civilian mission to space," the Shift4 Payments co-founder and CEO explains. "It's the first step towards a world where everyday people can go and explore among the stars."  https://t.co/lkYxf5PU17
"There a

In [32]:
df.head()

,tweet,date,time
0,alibaba plans up to billion u s dollar bond is...,2021-02-03,10:36:03
1,treasury is yellen to call regulator meeting o...,2021-02-03,09:59:33
2,amazon ceo jeff bezos is stepping down amazon ...,2021-02-03,07:19:16
3,amazon history how jeff bezos started with mai...,2021-02-03,06:41:11
4,jeff bezos email to employees keep inventing a...,2021-02-03,06:19:16


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 3 columns):
tweet    1000 non-null object
date     1000 non-null object
time     1000 non-null object
dtypes: object(3)
memory usage: 71.2+ KB


In [34]:
#Adding some new words to Vader Dictionary to judge stock market news better.

new_words = {'falls': -9, 'drops': -9, 'rise': 9, 'increases': 9, 'gain': 9, 'hiked': -9, 'dips': -9, 'declines': -9,
'decline': -9, 'hikes': -9, 'jumps': 9, 'lose': -9, 'profit': 9, 'loss': -9, 'shreds': -9, 'sell': -9, 'buy': 9, 'recession': -9,
'rupee weakens': -9, 'record low': -9, 'record high': 9, 'sensex up': 9, 'nifty down': -9, 'sensex down': -9, 'nifty up': 9}

analyser = SentimentIntensityAnalyzer()
analyser.lexicon.update(new_words)

scores = df['tweet'].apply(analyser.polarity_scores).tolist()
print(len(scores))
count = 1
for i in scores:
    print("count" + str(count) + str(i))
    count+= 1
scores_df = pd.DataFrame(scores)
scores_df.head()

1000
count1{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
count2{'neg': 0.0, 'neu': 0.847, 'pos': 0.153, 'compound': 0.2023}
count3{'neg': 0.0, 'neu': 0.779, 'pos': 0.221, 'compound': 0.34}
count4{'neg': 0.0, 'neu': 0.898, 'pos': 0.102, 'compound': 0.1779}
count5{'neg': 0.107, 'neu': 0.805, 'pos': 0.088, 'compound': -0.1124}
count6{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
count7{'neg': 0.174, 'neu': 0.758, 'pos': 0.068, 'compound': -0.743}
count8{'neg': 0.0, 'neu': 0.925, 'pos': 0.075, 'compound': 0.0772}
count9{'neg': 0.072, 'neu': 0.882, 'pos': 0.047, 'compound': -0.2415}
count10{'neg': 0.0, 'neu': 0.871, 'pos': 0.129, 'compound': 0.5267}
count11{'neg': 0.0, 'neu': 0.899, 'pos': 0.101, 'compound': 0.4215}
count12{'neg': 0.119, 'neu': 0.75, 'pos': 0.131, 'compound': 0.0516}
count13{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
count14{'neg': 0.0, 'neu': 0.918, 'pos': 0.082, 'compound': 0.1779}
count15{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
count

,neg,neu,pos,compound
0,0.000,1.000,0.000,0.0000
1,0.000,0.847,0.153,0.2023
2,0.000,0.779,0.221,0.3400
3,0.000,0.898,0.102,0.1779
4,0.107,0.805,0.088,-0.1124


In [35]:
df = pd.concat([df,scores_df],axis=1)
df.head(30)

,tweet,date,time,neg,neu,pos,compound
0,alibaba plans up to billion u s dollar bond is...,2021-02-03,10:36:03,0.000,1.000,0.000,0.0000
1,treasury is yellen to call regulator meeting o...,2021-02-03,09:59:33,0.000,0.847,0.153,0.2023
2,amazon ceo jeff bezos is stepping down amazon ...,2021-02-03,07:19:16,0.000,0.779,0.221,0.3400
3,amazon history how jeff bezos started with mai...,2021-02-03,06:41:11,0.000,0.898,0.102,0.1779
4,jeff bezos email to employees keep inventing a...,2021-02-03,06:19:16,0.107,0.805,0.088,-0.1124
5,rookisaacman is buying a spacex flight and pi...,2021-02-03,05:47:59,0.000,1.000,0.000,0.0000
6,there are certain industries that get attacke...,2021-02-03,05:42:24,0.174,0.758,0.068,-0.7430
7,senate vote clears path for biden is stimulus ...,2021-02-03,05:34:57,0.000,0.925,0.075,0.0772
8,seamless transition absolutely seamless trans...,2021-02-03,05:32:16,0.072,0.882,0.047,-0.2415
9,live now jared isaacman commander of the inspi...,2021-02-03,05:30:18,0.000,0.871,0.129,0.5267


In [36]:
df.set_index('date', inplace=True)
df.head()

,tweet,time,neg,neu,pos,compound
date,,,,,,
2021-02-03,alibaba plans up to billion u s dollar bond is...,10:36:03,0.000,1.000,0.000,0.0000
2021-02-03,treasury is yellen to call regulator meeting o...,09:59:33,0.000,0.847,0.153,0.2023
2021-02-03,amazon ceo jeff bezos is stepping down amazon ...,07:19:16,0.000,0.779,0.221,0.3400
2021-02-03,amazon history how jeff bezos started with mai...,06:41:11,0.000,0.898,0.102,0.1779
2021-02-03,jeff bezos email to employees keep inventing a...,06:19:16,0.107,0.805,0.088,-0.1124


In [37]:
df.to_csv('tweets/yahoo_financial_news_sentiments_with_scores.csv')